In [ ]:
!pip install tensorflow matplotlib numpy


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense


In [ ]:
# Vocabulary size (top words only)
max_features = 10000
maxlen = 200  # each review cut/padded to 200 words

print("Loading data...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

# Pad sequences to fixed length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


Loading data...
25000 train sequences
25000 test sequences


In [ ]:
# Vocabulary size (top words only)
max_features = 10000
maxlen = 200  # each review cut/padded to 200 words

print("Loading data...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

# Pad sequences to fixed length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


Loading data...
25000 train sequences
25000 test sequences


In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128))        # word embeddings
model.add(SimpleRNN(128, dropout=0.2))         # Simple RNN
model.add(Dense(1, activation='sigmoid'))      # output layer (binary)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=5,
                    validation_data=(x_test, y_test))


In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
# 🧠 IMDB Sentiment Classification using LSTM
import torch, torch.nn as nn, torch.optim as optim
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

# 1️⃣ Load & preprocess data
train_iter, test_iter = IMDB(split=('train', 'test'))
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for label, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
train_iter, test_iter = IMDB(split=('train', 'test'))

def process_batch(batch):
    texts = [torch.tensor(vocab(tokenizer(text))) for label, text in batch]
    labels = [1 if label == 'pos' else 0 for label, text in batch]
    return pad_sequence(texts, batch_first=True), torch.tensor(labels)

train_loader = DataLoader(list(train_iter), batch_size=32, shuffle=True, collate_fn=process_batch)
test_loader  = DataLoader(list(test_iter), batch_size=32, shuffle=False, collate_fn=process_batch)

# 2️⃣ Define model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 2)
    def forward(self, x):
        x = self.embedding(x)
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

model = LSTMClassifier(len(vocab), 64, 128).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3️⃣ Train model
train_acc, val_acc = [], []
for epoch in range(3):
    model.train()
    correct = total = loss_sum = 0
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        out = model(texts)
        loss = criterion(out, labels)
        loss.backward(); optimizer.step()
        loss_sum += loss.item()
        correct += (out.argmax(1) == labels).sum().item(); total += labels.size(0)
    train_accuracy = 100*correct/total; train_acc.append(train_accuracy)
    print(f"Epoch {epoch+1}: Loss={loss_sum/len(train_loader):.4f}, Train Acc={train_accuracy:.2f}%")

    # Validation
    model.eval(); correct = total = 0
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            preds = model(texts).argmax(1)
            correct += (preds == labels).sum().item(); total += labels.size(0)
    val_accuracy = 100*correct/total; val_acc.append(val_accuracy)
    print(f"Validation Accuracy: {val_accuracy:.2f}%\n")

# 4️⃣ Plot accuracy
plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
plt.xlabel('Epochs'); plt.ylabel('Accuracy (%)'); plt.title('LSTM Sentiment Classification')
plt.legend(); plt.show()

# 5️⃣ Example predictions
examples = [
    "This movie was amazing and the performances were outstanding!",
    "The film was boring, I did not like it at all.",
    "Not great, but not terrible either."
]
model.eval()
for text in examples:
    tokens = torch.tensor(vocab(tokenizer(text))).unsqueeze(0).to(device)
    pred = model(tokens).argmax(1).item()
    print(f"Review: {text}\nPredicted Sentiment: {'Positive 😊' if pred==1 else 'Negative 😞'}\n")


In [ ]:
# Train the model
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=5,
    validation_data=(x_test, y_test)
)

# Evaluate on test set
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
pip install torchtext